# Dataset Compiler

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display full data
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment

# Import data
data = pd.read_csv('data/compiled_data/main_data.csv')
data = data.drop(['Unnamed: 0'], axis=1)

In [2]:
data.shape

(3108, 171)

In [3]:
data.head()

fips    state   county  wins  poverty_rate_1960  poverty_rate_1970  \
0  1001  Alabama  Autauga     3               53.1               23.7   
1  1003  Alabama  Baldwin     3               41.8               22.6   
2  1005  Alabama  Barbour     3               65.7               43.7   
3  1007  Alabama     Bibb     3               56.8               38.2   
4  1009  Alabama   Blount     3               45.6               24.2   

   poverty_rate_1980  poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  \
0               16.6               15.7               10.9               11.6   
1               16.2               14.3               10.1               13.3   
2               30.8               25.2               26.8               26.1   
3               21.2               21.2               20.6               16.5   
4               18.0               15.3               11.7               14.7   

   population_1960  population_1970  population_1980  population_1990  \
0          18454.0          24313.0          32107.0          34009.0   
1          49722.0          58776.0          77726.0          97227.0   
2          24975.0          22424.0          24533.0          24785.0   
3          14775.0          13761.0          15520.0          16304.0   
4          25414.0          26746.0          36244.0          38924.0   

   population_2000  population_2010  persons_in_poverty_1960  \
0          43377.0            54255                   9807.0   
1         138148.0           180970                  20773.0   
2          26239.0            24530                  16395.0   
3          19852.0            21794                   8386.0   
4          50499.0            56993                  11595.0   

   persons_in_poverty_1970  persons_in_poverty_1980  persons_in_poverty_1990  \
0                   5758.0                   5337.0                   5324.0   
1                  13304.0                  12616.0                  13947.0   
2                   9788.0                   7559.0                   6244.0   
3                   5256.0                   3296.0                   3454.0   
4                   6462.0                   6527.0                   5938.0   

   persons_in_poverty_2000  persons_in_poverty_2010  \
0                   4738.0                     6267   
1                  14018.0                    24002   
2                   7032.0                     6395   
3                   4091.0                     3594   
4                   5930.0                     8396   

   rural_urban_continuum_code_2013  urban_influence_code_2013  metro_2013  \
0                              2.0                        2.0         1.0   
1                              3.0                        2.0         1.0   
2                              6.0                        6.0         0.0   
3                              1.0                        1.0         1.0   
4                              1.0                        1.0         1.0   

   civilian_labor_force_2000  civilian_labor_force_2001  \
0                      21720                      21955   
1                      69533                      69161   
2                      11373                      11250   
3                       8565                       9081   
4                      25106                      25305   

   civilian_labor_force_2002  civilian_labor_force_2003  \
0                      22094                      22604   
1                      69169                      72299   
2                      10971                      10977   
3                       8933                       8836   
4                      25757                      25900   

   civilian_labor_force_2004  civilian_labor_force_2005  \
0                      23218                    23949.0   
1                      74772                    76804.0   
2                      10633                    10760.0   
3                       8843    

## Drop Missing Values (data_dropna)
This sub-dataset will consist of all the values of the main dataset that do not have missing values. Rows that include missing values are dropped to create this sub-dataset. This might be useful for Random Forest Classifiers, that do not take missing values. For classifiers like SVM, however, missing values are not a problem so the main dataset would be preferred over this one.

In [4]:
data_dropna = data.copy()
data_dropna = data_dropna.dropna()
data_dropna.shape

(3021, 171)

In [5]:
data_dropna.to_csv('data/compiled_data/data_dropna.csv') # dropna data

## Interpolate for Missing Values (data_intp)
In this sub-dataset we use interpolation to fill in the missing values in the main dataset. To do this, we look at related columns, take the average of the two closest data points, and fill in the missing data. For example, if the column 'population_1980' had a missing value for one of the rows, we would take the average of 'population_1970' and 'population_1990' to fill in the value for 'population_1980'. Although this method does fill in some values, rows with large amounts of missing data could not be fixed, leaving some rows with missing data.

In [6]:
# Remove columns that cannot be interpolated

data_intp = data.copy()
data_intp = data_intp.drop(['rural_urban_continuum_code_2013', 
                       'urban_influence_code_2013', 
                       'metro_2013', 
                       'median_household_income_2019', 
                       'med_hh_income_percent_state_total_2019', 
                       'rural-urban_continuum_code_2003', 
                       'urban_influence_code_2003'], axis=1)

In [7]:
# Fill in missing values by taking chronological averages

first_year = [4, 10, 16, 22, 42, 62, 82, 102, 107, 112, 117, 122, 127, 132, 137]
last_year = [41, 61, 81, 101, 106, 111, 116, 121, 126, 131, 136, 141]

def isnan(x):
    return x != x

for i, v in enumerate(list(data_intp.isna().sum())):
    if v > 0:
        
        # Interpolate by taking the average of the value before and the value after
        if i not in first_year and i not in last_year:
            for row, value in enumerate(data_intp.iloc[:, i]):
                if isnan(data_intp.iloc[row, i]) == True:
                    if isnan(data_intp.iloc[row, i-1]) == False and isnan(data_intp.iloc[row, i+1]) == False:
                        middle = (int(data_intp.iloc[row, i-1]) + int(data_intp.iloc[row, i+1]))/2
                        data_intp.iloc[row, i] = middle
                        
        # Interpolate by making the value after the average of the three
        elif i in first_year:
            for row, value in enumerate(data_intp.iloc[:, i]):
                if isnan(data_intp.iloc[row, i]) == True:
                    if isnan(data_intp.iloc[row, i+1]) == False and isnan(data_intp.iloc[row, i+2]) == False:
                        left = 2*(int(data_intp.iloc[row, i+1])) - (int(data_intp.iloc[row, i+2]))
                        data_intp.iloc[row, i] = left
                        
        # Interpolate by making the value before the average of the three
        elif i in last_year:
            for row, value in enumerate(data_intp.iloc[:, i]):
                if isnan(data_intp.iloc[row, i]) == True:
                    if isnan(data_intp.iloc[row, i-1]) == False and isnan(data_intp.iloc[row, i-2]) == False:
                        right = 2*(int(data_intp.iloc[row, i-1])) - (int(data_intp.iloc[row, i-2]))
                        data_intp.iloc[row, i] = right

In [8]:
data.isna().sum() # before

fips                                                0
state                                               0
county                                              0
wins                                                0
poverty_rate_1960                                   0
poverty_rate_1970                                   0
poverty_rate_1980                                   0
poverty_rate_1990                                   0
poverty_rate_2000                                   0
poverty_rate_2010                                   0
population_1960                                    15
population_1970                                     7
population_1980                                     4
population_1990                                     2
population_2000                                     1
population_2010                                     0
persons_in_poverty_1960                            15
persons_in_poverty_1970                             7
persons_in_poverty_1980     

In [9]:
data_intp.isna().sum() #after

fips                                               0
state                                              0
county                                             0
wins                                               0
poverty_rate_1960                                  0
poverty_rate_1970                                  0
poverty_rate_1980                                  0
poverty_rate_1990                                  0
poverty_rate_2000                                  0
poverty_rate_2010                                  0
population_1960                                    7
population_1970                                    7
population_1980                                    4
population_1990                                    2
population_2000                                    1
population_2010                                    0
persons_in_poverty_1960                            7
persons_in_poverty_1970                            7
persons_in_poverty_1980                       

In [10]:
data_intp.shape

(3108, 164)

In [11]:
data_intp.to_csv('data/compiled_data/data_intp.csv') # interpolation data

## Use Proximity Averaging to Fill Missing Values (data_prox)
This sub-dataset is probably the most useful of the three covered so far. It does not have leftover missing values like data_intp and does not unnecessarily drop rows of valuable data. However, in constructing this sub-dataset we make an assumption: that counties within the same state will have aprroximately the same data. This means that when we fill in missing data, we are filling it in with the state averages, with respect to each county's population.

In [12]:
data_prox = data.copy()

In [13]:
# Proximity averaging does not work for D.C. because its a single district, we fill in missing data manually

data_prox.iloc[290, [0, 1, 2, 3, 22, 23, 24, 107, 108]]

fips                                                 11
state                              District of Columbia
county                                            Total
wins                                                  3
rural_urban_continuum_code_2013                     NaN
urban_influence_code_2013                           NaN
metro_2013                                          NaN
rural-urban_continuum_code_2003                       2
urban_influence_code_2003                             2
Name: 290, dtype: object

In [14]:
# INFO REGARDING AND FIXING MISSING DATA FOR D.C. FOR THE URBAN/METRO CODES

data_prox.iloc[290, 22] = 0 # District of Columbia Rural-Urban Continuum Code found on
# https://naldc.nal.usda.gov/download/CAT10407597/PDF
data_prox.iloc[290, 23] = data_prox.iloc[290, 108] # Urban Influence Code from 2003
data_prox.iloc[290, 24] = 1 # Usually metro_2013 is a 1 or 0

In [15]:
# Fill missing data with state averages

def average(num_list):
    total = 0
    for num in num_list:
        total += float(num)
    return total/len(num_list)

for i, v in enumerate(list(data_prox.isna().sum())):
    if v > 0:
        
        state = ''
        num_list = []
        
        for x in range(v):
            
            # Find the state
            for row, value in enumerate(data_prox.iloc[:, i]):
                if isnan(value) == True:
                    state = data_prox.iloc[row, 1]
                    break
                    
            # Append county values as a proportion of the county's population
            for row, value in enumerate(data_prox.loc[data_prox.state == state].iloc[:, i]):
                if isnan(value) == False:
                    
                    value = float(value)
                    
                    if '1960' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 10])
                    elif '1970' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 11])
                    elif '1980' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 12])
                    elif '1990' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 13])
                    elif '200' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 14])
                    elif '201' in data_prox.columns[i]:
                        num_list.append(value/data_prox.iloc[row, 15])
                    else:
                        num_list.append(value/data_prox.iloc[row, 15])

            state_average = 0        
            state_average_percent_pop = average(num_list)
            
            # The state average is determined by the state-wide average proportion times the county population
            if '1960' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 10]
            elif '1970' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 11]
            elif '1980' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 12]
            elif '1990' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 13]
            elif '200' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 14]
            elif '201' in data_prox.columns[i]:
                state_average = state_average_percent_pop * data_prox.iloc[row, 15]
            else:
                state_average = state_average_percent_pop * data_prox.iloc[row, 15]
            
            # Fill in the missing values
            for row, value in enumerate(data_prox.iloc[:, i]):
                if isnan(value) == True:
                    data_prox.iloc[row, i] = int(state_average)

In [16]:
data_prox.isna().sum() # there are no more missing values

fips                                               0
state                                              0
county                                             0
wins                                               0
poverty_rate_1960                                  0
poverty_rate_1970                                  0
poverty_rate_1980                                  0
poverty_rate_1990                                  0
poverty_rate_2000                                  0
poverty_rate_2010                                  0
population_1960                                    0
population_1970                                    0
population_1980                                    0
population_1990                                    0
population_2000                                    0
population_2010                                    0
persons_in_poverty_1960                            0
persons_in_poverty_1970                            0
persons_in_poverty_1980                       

In [17]:
data_prox.shape

(3108, 171)

In [18]:
data_prox.to_csv('data/compiled_data/data_prox.csv') # proximity averaged data

## Condensed Dataset (data_cnd)
This dataset is useful to classifiers that do not deal with noise easily. In previous datasets, 171 columns were used to describe each county. Some columns were more descriptive, others less important, and many of them inter-related. So, the data is condensed into 3108 by 34 dataframe, as opposed to the previous 3108 by 171 dataframe. Still, some of these columns are dependent on one another, so for future use even more columns may be removed to increase independence among the attributes.

In [19]:
data_cnd = pd.DataFrame()
data_cnd['fips'] = data_prox['fips']
data_cnd['state'] = data_prox['state']
data_cnd['county'] = data_prox['county']
data_cnd['wins'] = data_prox['wins']

In [20]:
# Condense columns from 171 columns to 34

data_cnd['poverty_rate'] = (data_prox.poverty_rate_1960 + data_prox.poverty_rate_1970 + data_prox.poverty_rate_1980 + data_prox.poverty_rate_1990 + data_prox.poverty_rate_2000 + data_prox.poverty_rate_2010)/6
data_cnd['population'] = (data_prox.population_1960 + data_prox.population_1970 + data_prox.population_1980 + data_prox.population_1990 + data_prox.population_2000 + data_prox.population_2010)/6
data_cnd['persons_in_poverty'] = (data_prox.persons_in_poverty_1960 + data_prox.persons_in_poverty_1970 + data_prox.persons_in_poverty_1980 + data_prox.persons_in_poverty_1990 + data_prox.persons_in_poverty_2000 + data_prox.persons_in_poverty_2010)/6
data_cnd['rural_urban_code'] = data_prox['rural_urban_continuum_code_2013']
data_cnd['urban_influence_code'] = data_prox['urban_influence_code_2013']
data_cnd['metro'] = data_prox['metro_2013']
data_cnd['civilian_labor_force'] = (data_prox.civilian_labor_force_2000 + data_prox.civilian_labor_force_2001 + data_prox.civilian_labor_force_2002 + data_prox.civilian_labor_force_2003 + data_prox.civilian_labor_force_2004 + data_prox.civilian_labor_force_2005 + data_prox.civilian_labor_force_2006 + data_prox.civilian_labor_force_2007 + data_prox.civilian_labor_force_2008 + data_prox.civilian_labor_force_2009 + data_prox.civilian_labor_force_2010 + data_prox.civilian_labor_force_2011 + data_prox.civilian_labor_force_2012 + data_prox.civilian_labor_force_2013 + data_prox.civilian_labor_force_2014 + data_prox.civilian_labor_force_2015 + data_prox.civilian_labor_force_2016 + data_prox.civilian_labor_force_2017 + data_prox.civilian_labor_force_2018 + data_prox.civilian_labor_force_2019)/20
data_cnd['employed'] = (data_prox.employed_2000 + data_prox.employed_2001 + data_prox.employed_2002 + data_prox.employed_2003 + data_prox.employed_2004 + data_prox.employed_2005 + data_prox.employed_2006 + data_prox.employed_2007 + data_prox.employed_2008 + data_prox.employed_2009 + data_prox.employed_2010 + data_prox.employed_2011 + data_prox.employed_2012 + data_prox.employed_2013 + data_prox.employed_2014 + data_prox.employed_2015 + data_prox.employed_2016 + data_prox.employed_2017 + data_prox.employed_2018 + data_prox.employed_2019)/20
data_cnd['unemployed'] = (data_prox.unemployed_2000 + data_prox.unemployed_2001 + data_prox.unemployed_2002 + data_prox.unemployed_2003 + data_prox.unemployed_2004 + data_prox.unemployed_2005 + data_prox.unemployed_2006 + data_prox.unemployed_2007 + data_prox.unemployed_2008 + data_prox.unemployed_2009 + data_prox.unemployed_2010 + data_prox.unemployed_2011 + data_prox.unemployed_2012 + data_prox.unemployed_2013 + data_prox.unemployed_2014 + data_prox.unemployed_2015 + data_prox.unemployed_2016 + data_prox.unemployed_2017 + data_prox.unemployed_2018 + data_prox.unemployed_2019)/20
data_cnd['unemployment_rate'] = (data_prox.unemployment_rate_2000 + data_prox.unemployment_rate_2001 + data_prox.unemployment_rate_2002 + data_prox.unemployment_rate_2003 + data_prox.unemployment_rate_2004 + data_prox.unemployment_rate_2005 + data_prox.unemployment_rate_2006 + data_prox.unemployment_rate_2007 + data_prox.unemployment_rate_2008 + data_prox.unemployment_rate_2009 + data_prox.unemployment_rate_2010 + data_prox.unemployment_rate_2011 + data_prox.unemployment_rate_2012 + data_prox.unemployment_rate_2013 + data_prox.unemployment_rate_2014 + data_prox.unemployment_rate_2015 + data_prox.unemployment_rate_2016 + data_prox.unemployment_rate_2017 + data_prox.unemployment_rate_2018 + data_prox.unemployment_rate_2019)/20
data_cnd['no_diploma'] = (data_prox.no_diploma_1970 + data_prox.no_diploma_1980 + data_prox.no_diploma_1990 + data_prox.no_diploma_2000 + data_prox['no_diploma_2014-2018'])/5
data_cnd['diploma_only'] = (data_prox.diploma_only_1970 + data_prox.diploma_only_1980 + data_prox.diploma_only_1990 + data_prox.diploma_only_2000 + data_prox['diploma_only_2014-2018'])/5
data_cnd['some_college/associate_degree'] = (data_prox.some_college_1970 + data_prox.some_college_1980 + data_prox['some_college/associate_degree_1990'] + data_prox['some_college/associate_degree_2000'] + data_prox['some_college/associate_degree_2014-2018'])/5
data_cnd['completed_college/bachelor_degree'] = (data_prox.completed_college_1970 + data_prox.completed_college_1980 + data_prox.bachelor_degree_or_higher_1990 + data_prox.bachelor_degree_or_higher_2000 + data_prox['bachelor_degree_or_higher_2014-2018'])/5
data_cnd['percent_no_diploma'] = (data_prox.percent_no_diploma_1970 + data_prox.percent_no_diploma_1980 + data_prox.percent_no_diploma_1990 + data_prox.percent_no_diploma_2000 + data_prox['percent_no_diploma_2014-2018'])/5
data_cnd['percent_diploma_only'] = (data_prox.percent_diploma_only_1970 + data_prox.percent_diploma_only_1980 + data_prox.percent_diploma_only_1990 + data_prox.percent_diploma_only_2000 + data_prox['percent_diploma_only_2014-2018'])/5
data_cnd['percent_some_college/associate_degree'] = (data_prox.percent_some_college_1970 + data_prox.percent_some_college_1980 + data_prox['percent_some_college/associate_degree_1990'] + data_prox['percent_some_college/associate_degree_2000'] + data_prox['percent_some_college/associate_degree_2014-2018'])/5
data_cnd['percent_completed_college/bachelor_degree'] = (data_prox.percent_completed_college_1970 + data_prox.percent_completed_college_1970 + data_prox['percent_bachelor_degree_or_higher_1990'] + data_prox['percent_bachelor_degree_or_higher_2000'] + data_prox['percent_bachelor_degree_or_higher_2014-2018'])/5

In [21]:
# Condense demographic data and COVID-19 data

columns = ['percent_white_male',
           'percent_white_female',
           'percent_black_male',
           'percent_black_female',
           'percent_native_american_male',
           'percent_native_american_female',
           'percent_asian_male',
           'percent_asian_female',
           'percent_hispanic_male',
           'percent_hispanic_female']

for i, v in enumerate(columns):
    c2010 = v + '_2010'
    c2019 = v + '_2019'
    for a, b in enumerate(data_prox[c2010]):
        data_prox.iloc[a, i+149] = float(data_prox.iloc[a, i+149])
    for a, b in enumerate(data_prox[c2019]):
        data_prox.iloc[a, i+159] = float(data_prox.iloc[a, i+159])
    data_cnd[v] = (data_prox[c2010] + data_prox[c2019])/2
    
data_cnd['cumulative_confirmed_cases'] = data_prox['cumulative_confirmed_cases']
data_cnd['cumulative_deaths'] = data_prox['cumulative_deaths']

In [22]:
data_cnd.head()

fips    state   county  wins  poverty_rate     population  \
0  1001  Alabama  Autauga     3     21.933333   34419.166667   
1  1003  Alabama  Baldwin     3     19.716667  100428.166667   
2  1005  Alabama  Barbour     3     36.383333   24581.000000   
3  1007  Alabama     Bibb     3     29.083333   17001.000000   
4  1009  Alabama   Blount     3     21.583333   39136.666667   

   persons_in_poverty  rural_urban_code  urban_influence_code  metro  \
0         6205.166667               2.0                   2.0    1.0   
1        16443.333333               3.0                   2.0    1.0   
2         8902.166667               6.0                   6.0    0.0   
3         4679.500000               1.0                   1.0    1.0   
4         7474.666667               1.0                   1.0    1.0   

   civilian_labor_force  employed  unemployed  unemployment_rate  no_diploma  \
0              24626.40  23330.15     1296.25              5.235      6002.6   
1              82407.40  77940.95     4466.45              5.415     17145.6   
2               9845.20   9039.75      805.45              8.185      6884.8   
3               8764.30   8183.45      580.85              6.620      4602.0   
4              25506.45  24173.90     1332.55              5.240     10024.8   

   diploma_only  some_college/associate_degree  \
0        7604.8                         5182.8   
1       22671.4                        19807.8   
2        4556.2                         2631.2   
3        3945.8                         1741.6   
4        8967.8                         5765.6   

   completed_college/bachelor_degree  percent_no_diploma  \
0                             4234.6               31.60   
1                            17325.4               30.72   
2                             1661.2               46.12   
3                              797.2               46.88   
4                             2313.2               42.66   

   percent_diploma_only  percent_some_college/associate_degree  \
0                 32.94                                  19.72   
1                 29.88                                  21.42   
2                 28.06                                  15.54   
3                 33.16                                  13.48   
4                 31.96                                  17.92   

   percent_completed_college/bachelor_degree  percent_white_male  \
0                                      14.60           38.196059   
1                                      16.84           42.724605   
2                                       9.90           26.530874   
3                                       6.34           39.330007   
4                                       6.92           47.467312   

   percent_white_female  percent_black_male  percent_black_female  \
0             39.605442            8.863984             10.070003   
1             44.808466            4.427916              4.699233   
2             23.516545           25.180116             22.500706   
3             37.344554           13.284798              8.390550   
4             48.740985            0.812518              0.745764   

   percent_native_american_male  percent_native_american_female  \
0                      0.217321                        0.257125   
1                      0.383858                        0.373638   
2                      0.377976                        0.268640   
3                      0.194704                        0.183642   
4                      0.321279                        0.304812   

   percent_asian_male  percent_asian_female  percent_hispanic_male  \
0            0.439203              0.591343               1.427005   
1            0.356886              0.551739               2.455862   
2            0.218840              0.220066               2.688941   
3            0.079719              0.075456               1.416059   
4            0.112748              0.156248               4.791606  

In [23]:
data_cnd.shape

(3108, 34)

In [24]:
data_cnd.to_csv('data/compiled_data/data_cnd.csv') # condensed data

## Non-Polarized Counties Dataset from 2000-2020 (data_2000_2020)

In [25]:
polar_counties_fips = pd.read_csv('data/raw_data/polar_counties_fips.csv')
polar_counties_name = pd.read_csv('data/raw_data/polar_counties_names.csv')

In [26]:
polar_counties_fips.head()

Unnamed: 0       0
0           0  1001.0
1           1  1003.0
2           2  1007.0
3           3  1009.0
4           4  1011.0

In [27]:
# Make the fips df have int values

polar_counties_fips_ints = []
for x in polar_counties_fips['0']:
    try:
        polar_counties_fips_ints.append(int(x))
    except:
        break

In [28]:
# Drop polarized counites from 2000-2020

data_2000_2020 = data_prox.copy()
drop_rows = []
for i, v in enumerate(data_cnd['fips']):
    if v in polar_counties_fips_ints:
        drop_rows.append(data_prox.index.values[i])

data_2000_2020 = data_2000_2020.drop(drop_rows)
data_2000_2020 = data_2000_2020.reset_index(drop=True)
data_2000_2020.head()

fips    state   county  wins  poverty_rate_1960  poverty_rate_1970  \
0  1005  Alabama  Barbour     3               65.7               43.7   
1  1023  Alabama  Choctaw     2               62.1               42.1   
2  1033  Alabama  Colbert     2               35.4               22.4   
3  1035  Alabama  Conecuh     3               63.7               45.0   
4  1071  Alabama  Jackson     2               52.1               24.4   

   poverty_rate_1980  poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  \
0               30.8               25.2               26.8               26.1   
1               31.5               30.2               24.5               20.2   
2               14.5               14.6               14.0               16.2   
3               29.4               29.7               26.6               30.4   
4               15.7               16.6               13.7               16.2   

   population_1960  population_1970  population_1980  population_1990  \
0          24975.0          22424.0          24533.0          24785.0   
1          18073.0          16515.0          16793.0          15929.0   
2          46794.0          49352.0          54165.0          51345.0   
3          18081.0          15545.0          15782.0          13939.0   
4          37307.0          39127.0          51132.0          47552.0   

   population_2000  population_2010  persons_in_poverty_1960  \
0          26239.0            24530                  16395.0   
1          15738.0            13668                  11225.0   
2          54384.0            53890                  16559.0   
3          13987.0            13119                  11515.0   
4          53333.0            52693                  19437.0   

   persons_in_poverty_1970  persons_in_poverty_1980  persons_in_poverty_1990  \
0                   9788.0                   7559.0                   6244.0   
1                   6958.0                   5285.0                   4809.0   
2                  11076.0                   7869.0                   7477.0   
3                   7000.0                   4642.0                   4145.0   
4                   9530.0                   8052.0                   7901.0   

   persons_in_poverty_2000  persons_in_poverty_2010  \
0                   7032.0                     6395   
1                   3858.0                     2758   
2                   7592.0                     8735   
3                   3719.0                     3982   
4                   7293.0                     8533   

   rural_urban_continuum_code_2013  urban_influence_code_2013  metro_2013  \
0                              6.0                        6.0         0.0   
1                              9.0                       10.0         0.0   
2                              3.0                        2.0         1.0   
3                              7.0                       11.0         0.0   
4                              6.0                        5.0         0.0   

   civilian_labor_force_2000  civilian_labor_force_2001  \
0                      11373                      11250   
1                       6184                       5869   
2                      25950                      25609   
3                       5483                       5274   
4                      27899                      27016   

   civilian_labor_force_2002  civilian_labor_force_2003  \
0                      10971                      10977   
1                       5731                       5651   
2                      24971                      24562   
3                       5163                       5149   
4                      26798                      27187   

   civilian_labor_force_2004  civilian_labor_force_2005  \
0                      10633                    10760.0   
1                       5462                     5300.0   
2                      24751                    24960.0   
3                       5061    

In [29]:
data_2000_2020.shape

(708, 171)

In [30]:
data_2000_2020.loc[data_2000_2020['wins'] == 6].shape # all 8 2020 bellwethers made it through!

(8, 171)

In [31]:
data_2000_2020.to_csv('data/segmented_data/data_2000_2020.csv') # 2000-2020 non-polarized counties dataset

## Non-Polarized Counties Condensed Dataset from 2000-2020 (data_cnd_2000_2020)

In [32]:
list(data_2000_2020.columns)

['fips',
 'state',
 'county',
 'wins',
 'poverty_rate_1960',
 'poverty_rate_1970',
 'poverty_rate_1980',
 'poverty_rate_1990',
 'poverty_rate_2000',
 'poverty_rate_2010',
 'population_1960',
 'population_1970',
 'population_1980',
 'population_1990',
 'population_2000',
 'population_2010',
 'persons_in_poverty_1960',
 'persons_in_poverty_1970',
 'persons_in_poverty_1980',
 'persons_in_poverty_1990',
 'persons_in_poverty_2000',
 'persons_in_poverty_2010',
 'rural_urban_continuum_code_2013',
 'urban_influence_code_2013',
 'metro_2013',
 'civilian_labor_force_2000',
 'civilian_labor_force_2001',
 'civilian_labor_force_2002',
 'civilian_labor_force_2003',
 'civilian_labor_force_2004',
 'civilian_labor_force_2005',
 'civilian_labor_force_2006',
 'civilian_labor_force_2007',
 'civilian_labor_force_2008',
 'civilian_labor_force_2009',
 'civilian_labor_force_2010',
 'civilian_labor_force_2011',
 'civilian_labor_force_2012',
 'civilian_labor_force_2013',
 'civilian_labor_force_2014',
 'civilian

In [33]:
data_cnd_2000_2020 = pd.DataFrame()

In [34]:
data_cnd_2000_2020.shape

(0, 0)

In [35]:
keep = [
 'fips',
 'state',
 'county',
 'wins',
 'poverty_rate_1960',
 'poverty_rate_2010',
 'population_1960',
 'population_2010',
 'urban_influence_code_2013',
 'civilian_labor_force_2000',
 'civilian_labor_force_2019',
 'unemployment_rate_2000',
 'unemployment_rate_2019',
 'median_household_income_2019',
 'med_hh_income_percent_state_total_2019',
 'urban_influence_code_2003',
 'percent_no_diploma_1970',
 'percent_no_diploma_2014-2018',
 'percent_diploma_only_1970',
 'percent_diploma_only_2014-2018',
 'percent_some_college_1970',
 'percent_some_college/associate_degree_2014-2018',
 'percent_completed_college_1970',
 'percent_bachelor_degree_or_higher_2014-2018',
 'percent_white_male_2019',
 'percent_white_female_2019',
 'percent_black_male_2019',
 'percent_black_female_2019',
 'percent_native_american_male_2019',
 'percent_native_american_female_2019',
 'percent_asian_male_2019',
 'percent_asian_female_2019',
 'percent_hispanic_male_2019',
 'percent_hispanic_female_2019',
 'cumulative_confirmed_cases',
 'cumulative_deaths'
]
for i, v in enumerate(keep):
    data_cnd_2000_2020[v] = data_2000_2020[v]

In [36]:
data_cnd_2000_2020.head()

fips    state   county  wins  poverty_rate_1960  poverty_rate_2010  \
0  1005  Alabama  Barbour     3               65.7               26.1   
1  1023  Alabama  Choctaw     2               62.1               20.2   
2  1033  Alabama  Colbert     2               35.4               16.2   
3  1035  Alabama  Conecuh     3               63.7               30.4   
4  1071  Alabama  Jackson     2               52.1               16.2   

   population_1960  population_2010  urban_influence_code_2013  \
0          24975.0            24530                        6.0   
1          18073.0            13668                       10.0   
2          46794.0            53890                        2.0   
3          18081.0            13119                       11.0   
4          37307.0            52693                        5.0   

   civilian_labor_force_2000  civilian_labor_force_2019  \
0                      11373                       8537   
1                       6184                       4670   
2                      25950                      23646   
3                       5483                       4533   
4                      27899                      23484   

   unemployment_rate_2000  unemployment_rate_2019  \
0                     5.5                     3.8   
1                     6.5                     4.7   
2                     5.5                     3.7   
3                     6.5                     4.0   
4                     5.2                     3.3   

   median_household_income_2019  med_hh_income_percent_state_total_2019  \
0                         35972                                    69.5   
1                         39808                                    76.9   
2                         46683                                    90.2   
3                         41539                                    80.2   
4                         44322                                    85.6   

   urban_influence_code_2003  percent_no_diploma_1970  \
0                        5.0                     68.8   
1                        1.0                     68.2   
2                        5.0                     56.3   
3                        2.0                     71.2   
4                        2.0                     71.5   

   percent_no_diploma_2014-2018  percent_diploma_only_1970  \
0                          27.0                       19.0   
1                          18.6                       22.9   
2                          15.6                       29.9   
3                          19.2                       20.7   
4                          17.5                       21.0   

   percent_diploma_only_2014-2018  percent_some_college_1970  \
0                            35.7                        4.9   
1                            38.2                        4.5   
2                            36.0                        8.0   
3                            48.8                        4.0   
4                            40.8                        4.1   

   percent_some_college/associate_degree_2014-2018  \
0                                             25.1   
1                                             30.2   
2                                             29.7   
3                                             21.3   
4                                             27.5   

   percent_completed_college_1970  \
0                             7.3   
1                             4.3   
2                             5.8   
3                             4.1   
4                             3.4   

   percent_bachelor_degree_or_higher_2014-2018  percent_white_male_2019  \
0                                         12.2                25.881066   
1                                         13.0                27.794106   
2                                         18.7                38.782788   
3                                         10.7                25.714759   
4                 

In [37]:
data_cnd_2000_2020.shape

(708, 36)

In [38]:
data_cnd_2000_2020.to_csv('data/segmented_data/data_cnd_2000_2020.csv') # 2000-2020 non-polarized counties condensed dataset